In [2]:
pip install selenium


                                              0.0/10.2 MB ? eta -:--:--
                                             0.0/10.2 MB 960.0 kB/s eta 0:00:11
                                              0.1/10.2 MB 1.7 MB/s eta 0:00:06
     -                                        0.3/10.2 MB 2.4 MB/s eta 0:00:05
     --                                       0.5/10.2 MB 3.2 MB/s eta 0:00:04
     ---                                      0.9/10.2 MB 4.3 MB/s eta 0:00:03
     -----                                    1.4/10.2 MB 5.3 MB/s eta 0:00:02
     -------                                  1.9/10.2 MB 6.2 MB/s eta 0:00:02
     ---------                                2.4/10.2 MB 6.7 MB/s eta 0:00:02
     -----------                              2.9/10.2 MB 7.0 MB/s eta 0:00:02
     -------------                            3.3/10.2 MB 7.3 MB/s eta 0:00:01
     ---------------                          3.9/10.2 MB 7.5 MB/s eta 0:00:01
     ----------------                         4.2/10.2 MB

In [3]:
from selenium import webdriver  # Automate web browser control
from selenium.webdriver.edge.options import Options  # Options for Edge browser
from selenium.webdriver.common.by import By  # Search for elements on the web page
from selenium.webdriver.support.ui import WebDriverWait  # Wait for a certain condition before the next operation
from selenium.webdriver.support import expected_conditions as EC  # Expected conditions in Selenium
import pandas as pd  
from datetime import datetime  
import time  # Library to create delay during execution

#### FED rate

In [112]:
# parameter
start = datetime(2010,1,4)
end = datetime(2023,11,3)
link = 'https://www.investing.com/economic-calendar/interest-rate-decision-168/'

In [113]:
# access to the link using edge web driver
options = Options()
options.add_experimental_option('detach',True)
driver = webdriver.Edge(options=options)
driver.get(link)

# wait 
time.sleep(5)

In [114]:
# find showMore button by Xpath
show_more = driver.find_element(By.XPATH,'//*[@id="showMoreHistory168"]') 

In [115]:

# close sign up pop up 
driver.find_element(By.XPATH,'//*[@id="PromoteSignUpPopUp"]/div[2]/i').click()
time.sleep(2)

while True:
    
    # find list of rows in the table
    patial = driver.find_elements(By.XPATH,'//*[starts-with(@id, "historicEvent_")]/td[1]') 

    # the last row in the table
    yesterday = datetime.strptime(patial[-1].text, '%b %d, %Y')
    if yesterday <= start:
        break # loop until it's find the start date

    show_more.click()
    time.sleep(2)



In [118]:
date = driver.find_elements(By.XPATH,' //*[starts-with(@id, "historicEvent_")]/td[1]')
rate = driver.find_elements(By.XPATH,' //*[starts-with(@id, "historicEvent_")]/td[3]')

data_len = len(date)

save = {
    'date':[],
    'FED':[],
}

for day in range(data_len):
    curr_day=datetime.strptime(date[day].text, '%b %d, %Y')
    if curr_day > end:
        continue
    numerical_form = curr_day.strftime('%d/%m/%Y')          
    save['date'].append(numerical_form)
    save['FED'].append(float(rate[day].text[:-1]))

df = pd.DataFrame(save)

In [119]:
df

,date,FED
0,01/11/2023,5.50
1,20/09/2023,5.50
2,26/07/2023,5.50
3,14/06/2023,5.25
4,03/05/2023,5.25
...,...,...
108,23/06/2010,0.25
109,28/04/2010,0.25
110,16/03/2010,0.25
111,27/01/2010,0.25


In [121]:
df.to_csv('FED.csv')

#### GOLD

In [83]:
url = 'https://finance.yahoo.com/quote/GC%3DF/history?p=GC%3DF'
start_date_input = datetime(2010,1,4)
end_date_input = datetime(2023,11,3)

In [108]:

# Step 1: set def with parameters
def get_data(url, start_date_input, end_date_input):
    
    options = webdriver.EdgeOptions()  # create an EdgeOptions object
    options.add_experimental_option('detach', True)  # add the 'detach' option - website runs in detached mode
    driver = webdriver.Edge(options=options)  # create a webdrive object using the Edge browser and pass it the options object
    driver.get(url)
    print('Finish step 1!')
    
    # Step 2: set range date 
    def set_date_and_apply(start_date, end_date):
        time.sleep(5)
        date_picker = driver.find_element(By.XPATH, '//*[@id="Col1-1-HistoricalDataTable-Proxy"]/section/div[1]/div[1]/div[1]/div/div/div[1]')
        date_picker.click()

        # Convert datetime objects to strings
        start_date_str = start_date.strftime('%d-%m-%Y')
        end_date_str = end_date.strftime('%d-%m-%Y')

        start_input = driver.find_element(By.XPATH, '//*[@id="dropdown-menu"]/div/div[1]/input')
        start_input.clear()
        start_input.send_keys(start_date_str)

        end_input = driver.find_element(By.XPATH, '//*[@id="dropdown-menu"]/div/div[2]/input')
        end_input.clear()
        end_input.send_keys(end_date_str)

        apply_button = driver.find_element(By.XPATH, '//*[@id="dropdown-menu"]/div/div[3]/button[1]')
        apply_button.click()

        apply_button = driver.find_element(By.XPATH, '//*[@id="Col1-1-HistoricalDataTable-Proxy"]/section/div[1]/div[1]/button')
        apply_button.click()

    set_date_and_apply(start_date_input, end_date_input)
    print('Finish step 2!')
    
    # Step3: Scroll down to download all data, (if you haven't scrolled down yet, the data will only load up to 100 lines)
    
    time.sleep(5)  # Wait for initial content to load
    previous_length = 0

    while True:
        # Scroll down to the bottom of the table
        bottom = driver.find_element(By.XPATH, '//*[@id="Col1-1-HistoricalDataTable-Proxy"]/section/div[2]/table/tfoot/tr')
        driver.execute_script("arguments[0].scrollIntoView(true);", bottom)
        time.sleep(5)  # Wait for the webpage to update after each scroll

        # Check if the number of rows has changed
        current_length = len(driver.find_elements(By.XPATH, '//*[@id="Col1-1-HistoricalDataTable-Proxy"]/section/div[2]/table/tbody/tr'))
        if current_length == previous_length:
            break  # If no more data is loaded, exit the loop
        else:
            previous_length = current_length  # Update the previous length to the current length

    # Retrieve all rows from the table
    rows = driver.find_elements(By.XPATH, '//*[@id="Col1-1-HistoricalDataTable-Proxy"]/section/div[2]/table/tbody/tr')
    print('Finish step 3!')
    
    # Step 4: crawl rows, (avoid index out of the range, when collect for col)
    data = []   # list save
    for row in rows:
        cells = row.find_elements(By.TAG_NAME, 'td')     
        # find_elements, find all cells (td tags in HTML). find_elements(By.TAG_NAME, ...), find_elements(By.XPATH, ...)


        if len(cells) >= 5:

            date_text = cells[0].text
            open_text = cells[1].text
            high_text = cells[2].text
            low_text = cells[3].text
            close_text = cells[4].text    # close ở cột cells[4]

            try:
                    f = lambda x: float(x.replace(',', ''))
                    open_, high_, low_, close_ = f(open_text), f(high_text), f(low_text), f(close_text)


                    numerical_date = datetime.strptime(date_text, '%b %d, %Y').strftime('%d/%m/%Y')
                    # datetime - Convert STRP string to STRF specific date %Y-%m-%d, or %d-%m-%Y, ... (for example from 'Jan 01, 2020' to ' 2020-01-01').
                    data.append({'date': numerical_date,
                                 'Open': open_,
                                 'High': high_,
                                 'Low': low_,
                                 'Close': close_})
            except:
                print('Error:', date_text)
    print('Finish step 4!')
    
    df = pd.DataFrame(data)
    
    return df

In [109]:
start = datetime(2010,1,4)
end = datetime(2023,11,20)
link = 'https://finance.yahoo.com/quote/GC%3DF/history?p=GC%3DF'
df = get_data(link, start, end)

Finish step 1!
Finish step 2!
Finish step 3!
Error: Jan 29, 2018
Error: Feb 15, 2016
Error: Jan 18, 2016
Error: Mar 28, 2013
Finish step 4!


In [110]:
df

,date,Open,High,Low,Close
0,24/11/2023,1991.2,2004.5,1990.9,2003.0
1,22/11/2023,1999.1,1999.4,1991.0,1991.4
2,21/11/2023,1990.7,2005.5,1988.5,1999.3
3,20/11/2023,1978.7,1979.4,1969.7,1977.7
4,17/11/2023,1982.5,1984.0,1979.8,1981.6
...,...,...,...,...,...
3429,07/04/2010,1152.2,1152.3,1144.0,1152.2
3430,06/04/2010,1152.3,1153.0,1132.6,1152.3
3431,05/04/2010,1130.8,1138.5,1122.7,1135.1
3432,04/04/2010,1122.3,1133.3,1119.9,1132.9


In [111]:
df.to_csv('Gold.csv')